In [ ]:
library(dplyr)
library(readr)
install.packages("caret")
library(caret)
install.packages("ggplot2")
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [ ]:
# Instalar y cargar las bibliotecas necesarias
install.packages("googledrive")
library(googledrive)

In [ ]:
# Autenticarse con Google Drive
drive_auth()

In [ ]:
# Especificar el ID del archivo y la ruta de destino
file_id <- "https://drive.google.com/file/d/1WDRKvrZrvSlQa8qNgUCoGwEyepYNm3dz/view?usp=drivesdk"  # Reemplaza este ID con el ID de tu archivo
drive_download(as_id(file_id), path = "archivo.csv", overwrite = TRUE)

In [ ]:
# Leer el archivo CSV desde la URL
urlAge <- "https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/machine_age.csv"
urlMechanic <- "https://raw.githubusercontent.com/cesargar1507/DatasetsUB/main/mechanic_antiquity.csv"
df <- read_csv("archivo.csv", show_col_types = FALSE)
df_age <- read_csv(urlAge, show_col_types = FALSE)
df_mechanic <- read_csv(urlMechanic, show_col_types = FALSE)

In [ ]:
summary(df)

In [ ]:
head(df,5)

**LIMPIEZA DE DATOS**

In [ ]:
sapply(df, function(x) sum(is.na(x)))
#Fiabilidad suficiente para continuar con el trabajo

In [ ]:
# Remover el símbolo de dólar y las comas, y convertir a float
df$parts_cost <- as.numeric(gsub("[\\$,()]", "", df$parts_cost))

In [ ]:
# Crear el boxplot de parts_cost
ggplot(df, aes(y = parts_cost)) +
  geom_boxplot(fill = 'skyblue', color = 'black') +
  labs(title = 'Boxplot de parts cost',
       y = 'Costo de partes') +
  theme_minimal()
#Hacer rangos, 1 desv tip, 2 desp tipica, etc...

In [ ]:
# Convertir la columna 'Date' al formato adecuado
df$date <- as.Date(df$date, format="%m/%d/%Y")  # Ajusta el formato según tus datos
# Ordenar el DataFrame por fecha
df <- df %>%
  arrange(date)

In [ ]:
# Histograma de 'downtimeGross'
hist_downtimeGross <- ggplot(df, aes(x = downtime_gross)) +
  geom_histogram(binwidth = 0.1, fill = "green", color = "black", alpha = 0.7) +
  labs(title = "Histograma de Downtime Bruto",
       x = "Downtime Bruto (horas)",
       y = "Frecuencia") +
  theme_minimal()

# Mostrar histogramas
print(hist_downtimeGross)

In [ ]:
mean(df$downtime_gross < 0)

In [ ]:
# Filtrar registros donde downtimeGross no sea negativo
df_filtered <- df %>%
  filter(downtime_gross >= 0)

In [ ]:
mean(df$repair_hours < 0)

In [ ]:
# Calcular valor absoluto de repair hours
df$repair_hours <- abs(df$repair_hours)
#Quitarlo poco porcentaje de datos

In [ ]:
#Eliminar columnas que no seran utilizadas
df <- select(df, -problem_code, -failure_code, -cause_code, -action_code, -pit_coverage, -real_downtime, -waiting_hours)
#Se han eliminado codigos y variables calculadas

In [ ]:
# Filtrar el DataFrame por 'WO Type' igual a 'Corrective Maintenance'
df <- df %>%
  filter(`wo_type` == "Corrective Maintenance")
#Colocar comentario de por qué el filtro, colocar que tecnicas pueden aplicarse y cuales no, colocar en apartado diferente

In [ ]:
# Calcular la diferencia en días entre fechas consecutivas
df <- df %>%
  group_by(equipment) %>%
  mutate(Days_Between_Failures = difftime(date, lag(date), units = "days"))

# Reemplazar los valores NA en Days_Between_Failures
df$Days_Between_Failures[is.na(df$Days_Between_Failures)] <- as.numeric(difftime(df$date[is.na(df$Days_Between_Failures)], as.Date("2023-01-01"), units = "days"))

In [ ]:
# Convertir 'Days_Between_Failures' a numeric
df$Days_Between_Failures <- as.numeric(df$Days_Between_Failures, units = "days")

In [ ]:
#Cambio de nombre de mecánico cuyo nombre está escrito de dos formas
df <- df %>%
  mutate(mechanic = ifelse(mechanic == "GALEAS PEREZ EDWIN ROGELIO", "Edwin Rogelio Galeas Perez", mechanic))

**ANALISIS DE VARIABLES**

In [ ]:
dim(df)

In [ ]:
# Calcular la frecuencia de cada operación
operation_freq <- df %>%
  group_by(operation) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular el porcentaje acumulado
operation_freq <- operation_freq %>%
  mutate(cum_count = cumsum(count),
         cum_perc = cum_count / sum(count))

operation_freq

In [ ]:
# Crear el gráfico de barras
ggplot(df, aes(y = problem_desc)) +
  geom_bar(fill = 'skyblue', color = 'black') +
  labs(title = 'Frecuencia de Variables Categóricas',
       x = 'Frecuencia',
       y = 'Categoria') +
  theme_minimal()
#Ordenar, es util frecuencia acumulada top 3

In [ ]:
# Crear el gráfico de barras
ggplot(df, aes(y = failure_desc)) +
  geom_bar(fill = 'skyblue', color = 'black') +
  labs(title = 'Frecuencia de Variables Categóricas',
       x = 'Frecuencia',
       y = 'Categoria') +
  theme_minimal()
#Ordenar, es util frecuencia acumulada top 3
#Colocar porcentaje a la derecha

In [ ]:
# Calcular la frecuencia de causas de fallo
cause_freq <- df %>%
  group_by(cause_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular el porcentaje acumulado
cause_freq <- cause_freq %>%
  mutate(cum_count = cumsum(count),
         cum_perc = cum_count / sum(count))
cause_freq

In [ ]:
# Calcular la frecuencia de acciones correctivas
action_freq <- df %>%
  group_by(action_desc) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular el porcentaje acumulado
action_freq <- action_freq %>%
  mutate(cum_count = cumsum(count),
         cum_perc = cum_count / sum(count))
action_freq
#Grafica de distribución

In [ ]:
# Calcular la frecuencia de acciones correctivas
mechanic_freq <- df %>%
  group_by(mechanic) %>%
  summarise(count = n()) %>%
  arrange(desc(count))

# Calcular el porcentaje acumulado
mechanic_freq <- mechanic_freq %>%
  mutate(cum_count = cumsum(count),
         cum_perc = cum_count / sum(count))
mechanic_freq

In [ ]:
# Crear el boxplot de repair_hours
ggplot(df, aes(y = repair_hours)) +
  geom_boxplot(fill = 'skyblue', color = 'black') +
  labs(title = 'Boxplot de repair hours',
       y = 'Costo de partes') +
  theme_minimal()
#Replicar gráfico con y sin outliers, analizar la frecuencia por rangos, medir con desviación típica, intentar medir cada cuanto ocurre el outlier, recomendación a añadir

**ANALISIS DE VARIABLE OBJETIVO "DOWNTIME GROSS"**

In [ ]:
# Histograma de 'downtimeGross'
hist_downtimeGross <- ggplot(df, aes(x = downtime_gross)) +
  geom_histogram(binwidth = 0.1, fill = "green", color = "black", alpha = 0.7) +
  labs(title = "Histograma de Downtime Bruto",
       x = "Downtime Bruto (horas)",
       y = "Frecuencia") +
  theme_minimal()

# Mostrar histogramas
print(hist_downtimeGross)
#Hacerlo más grande

In [ ]:
# Crear el boxplot de daysbetween_failures
ggplot(df, aes(y = downtime_gross)) +
  geom_boxplot(fill = 'skyblue', color = 'black') +
  labs(title = 'Boxplot de Days Between Failures',
       y = 'Días entre Fallas') +
  theme_minimal()

**MODELO DE REGRESION LINEAL INICIAL PARA PREDECIR DOWNTIME**



In [ ]:
df <- select(df, -wo_type)
#Comparar con un glm

In [ ]:
head(df,5)

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir la proporción de entrenamiento
train_proportion <- 0.80

# Crear índices de muestreo para el conjunto de entrenamiento
trainIndex <- createDataPartition(df$downtime_gross, p = train_proportion, list = FALSE)

# Crear conjunto de entrenamiento
train_data1 <- df[trainIndex,]

# Crear conjunto de prueba
test_data1 <- df[-trainIndex,]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data1), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data1), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
#Dividir en entrenamiento, test, validación por ejemplo 70,20,10
model1 <- glm(downtime_gross~. -equipment -downtime_gross -repair_hours -Days_Between_Failures -date -changed_pcs, data = train_data1)
# Resumen del modelo
summary(model1)

**CALCULO DE VARIABLES PARA MEJORA DEL AJUSTE**

In [ ]:
#Calcular los fallos acumulativos
df <- df %>%
  group_by(equipment) %>%
  mutate(Cumulative_Failure_Count = row_number())

In [ ]:
# Calcular la mediana de tiempo de reparación
df <- df %>%
  group_by(equipment) %>%
  mutate(Median_Repair_Time = median(repair_hours, na.rm = TRUE))

In [ ]:
# Calcular el costo acumulado de mantenimiento
df <- df %>%
  group_by(equipment) %>%
  mutate(Cumulative_Maintenance_Cost = cumsum(parts_cost))

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir la proporción de entrenamiento
train_proportion <- 0.80

# Crear índices de muestreo para el conjunto de entrenamiento
trainIndex <- createDataPartition(df$downtime_gross, p = train_proportion, list = FALSE)

# Crear conjunto de entrenamiento
train_data2 <- df[trainIndex,]

# Crear conjunto de prueba
test_data2 <- df[-trainIndex,]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data2), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data2), "\n")

**MODELO DE REGRESION LINEAL PARA PREDECIR DOWNTIME CON MAS VARIABLES**

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
model2 <- glm(downtime_gross~. -equipment -downtime_gross -repair_hours -date -changed_pcs, data = train_data2)
# Resumen del modelo
summary(model2)

**MODELO AGREGANDO LA VARIABLE "MACHINE AGE"**

In [ ]:
# Renombrar la columna 'Equipment' a 'equipment' en df_age
df_age <- df_age %>% rename(equipment = Equipment)

In [ ]:
df_merged <- df %>%
  left_join(df_age %>% select(equipment, Age), by = "equipment")

In [ ]:
head(df_merged,3)

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir la proporción de entrenamiento
train_proportion <- 0.80

# Crear índices de muestreo para el conjunto de entrenamiento
trainIndex <- createDataPartition(df_merged$downtime_gross, p = train_proportion, list = FALSE)

# Crear conjunto de entrenamiento
train_data3 <- df_merged[trainIndex,]

# Crear conjunto de prueba
test_data3 <- df_merged[-trainIndex,]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data3), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data3), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
model3 <- glm(downtime_gross~. -equipment -downtime_gross -repair_hours -date -changed_pcs, data = df_train3)
# Resumen del modelo
summary(model3)

**MODELO AGREGANDO LA VARIABLE "MECHANIC ANTIQUITY"**

In [ ]:
df_merged2 <- df_merged %>%
  left_join(df_mechanic %>% select(mechanic, antiquity), by = "mechanic")

In [ ]:
# Establecer una semilla para reproducibilidad
set.seed(123)

# Definir la proporción de entrenamiento
train_proportion <- 0.80

# Crear índices de muestreo para el conjunto de entrenamiento
trainIndex <- createDataPartition(df_merged2$downtime_gross, p = train_proportion, list = FALSE)

# Crear conjunto de entrenamiento
train_data4 <- df_merged2[trainIndex,]

# Crear conjunto de prueba
test_data4 <- df_merged2[-trainIndex,]

# Comprobar las dimensiones de cada conjunto
cat("Dimensiones del conjunto de entrenamiento: ", dim(train_data4), "\n")
cat("Dimensiones del conjunto de prueba: ", dim(test_data4), "\n")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
model4 <- glm(downtime_gross~. -equipment -downtime_gross -repair_hours -date -changed_pcs, data = train_data4)
# Resumen del modelo
summary(model4)

**AGREGANDO LA VARIABLE "CHANGED_PCS"**

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
model5 <- glm(downtime_gross~. -equipment -downtime_gross -repair_hours -date -antiquity, data = train_data4)
# Resumen del modelo
summary(model5)

Variables no significativas hay que eliminarlas del modelo

**IMPLEMENTACION DE BACKWARD**

In [ ]:
sapply(df_merged2, function(x) sum(is.na(x)))

equipment                   operation 
                          0                           0 
                 base_model                problem_desc 
                          0                           0 
               failure_desc                  cause_desc 
                          0                           0 
                action_desc                    mechanic 
                          0                           0 
                       date              downtime_gross 
                          0                           0 
                 parts_cost                repair_hours 
                          0                           0 
                changed_pcs       Days_Between_Failures 
                          0                           0 
   Cumulative_Failure_Count          Median_Repair_Time 
                          0                           0 
Cumulative_Maintenance_Cost                         Age 
                          0                           0 
                  antiquity 
                          0

In [ ]:
# Realizar backward elimination
model_null <- lm(downtime_gross ~ 1, data = df_merged2)
model_forward <- step(model_null,
                      scope = list(lower = model_null, upper = model5),
                      direction = "forward")
model_forward$anova

In [ ]:
formula = as.formula("Days_Between_Failures ~ operation + base_model + mechanic + downtime_gross +
    parts_cost + Cumulative_Failure_Count + Median_Repair_Time +
    Cumulative_Maintenance_Cost + Age")

In [ ]:
# Inicializar y ajustar el modelo de regresión lineal
model6 <- lm(formula = formula, data = df_merged3)
# Resumen del modelo
summary(model6)